In [1]:
import numpy as np
import pandas as pd

In [2]:
class SarsaLambdaTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9, trace_decay=0.9):
        self.actions = actions
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon = e_greedy
        self.table = pd.DataFrame(columns=self.actions, dtype=np.float64)
        self.lambda_ = trace_decay
        self.eligibility_trace = self.table.copy()

    def check_state_exist(self, state):
        if state not in self.table.index:
            s = pd.Series(
                [0] * len(self.actions),
                index=self.table.columns,
                name=state
            )
            self.table = self.table.append(s)
            self.eligibility_trace = self.eligibility_trace.append(s)
    
    def choose_action(self, observation):
        # q_table 表中不存在就添加到表中
        self.check_state_exist(observation)
        # action selection
        if np.random.uniform() < self.epsilon:
            # 提取要操作的表中的某一行
            state_action = self.table.loc[observation, :]
            # 获取数据最大的那列的名字
            action = np.random.choice(
                state_action[state_action == np.max(state_action)].index
            )
        else:
            action = np.random.choice(self.actions)
        return action
    
    def learn(self, s, a, r, s_, a_):
        self.check_state_exist(s_)
        predict = self.table.loc[s, a]
        if s_ != 'terminal':
            target = r + self.gamma * self.table.loc[s_, a_]
        else:
            target = r
        error = target - predict

        #Method 1
        # self.eligibility_trace.loc[s, a] += 1

        #Method 2
        self.eligibility_trace.loc[s, :] = 0
        self.eligibility_trace.loc[s, a] = 1 

        # Q update
        self.table += self.lr * error * self.eligibility_trace

        # decay eligibility trace after update
        self.eligibility_trace *= self.gamma * self.lambda_
    
    def get_table(self):
        return self.table

In [5]:
def update(env, RL):
    def __update():
        for episode in range(100):
            # initial observation
            observation = env.reset()
            action = RL.choose_action(str(observation))
            while True:
                # fresh env
                env.render()
                
                # RL take action and get next observation and reward
                observation_, reward, done = env.step(action)

                # RL choose action based on observation
                action_ = RL.choose_action(str(observation_))
                
                # RL learn from this transition
                RL.learn(str(observation), action, reward, str(observation_), action_)
                
                # swap observation
                observation = observation_

                action = action_
                
                if done:
                    break
        
        print(RL.get_table())
        # end of game
        print('game over')
        env.destroy()
    return __update

In [6]:
from maze_env import Maze
env = Maze()
RL = SarsaLambdaTable(actions=list(range(env.n_actions)))
env.after(20, update(env, RL))
env.mainloop()

                                         0         1             2         3
[5.0, 5.0, 35.0, 35.0]        1.580754e-03  0.187176  2.319580e-03  0.002752
[5.0, 45.0, 35.0, 75.0]       1.364543e-03  0.221277 -2.320020e-02 -0.001743
[45.0, 5.0, 75.0, 35.0]      -1.041214e-03 -0.013651 -2.205749e-02  0.017363
[45.0, 45.0, 75.0, 75.0]     -3.383466e-03 -0.040573 -1.990228e-02  0.004024
terminal                      0.000000e+00  0.000000  0.000000e+00  0.000000
[5.0, 85.0, 35.0, 115.0]     -3.590606e-04  0.303118 -8.648015e-02  0.001662
[5.0, 125.0, 35.0, 155.0]    -6.551561e-03  0.001072  3.655094e-01  0.004587
[45.0, 125.0, 75.0, 155.0]   -1.919202e-02  0.001368  4.581667e-01  0.002023
[85.0, 5.0, 115.0, 35.0]     -3.570454e-04 -0.048982 -1.485809e-03 -0.000150
[125.0, 5.0, 155.0, 35.0]    -2.516212e-05 -0.002759 -2.537675e-05 -0.008101
[125.0, 45.0, 155.0, 75.0]   -2.354480e-09  0.002298 -1.215553e-08 -0.009998
[125.0, 85.0, 155.0, 115.0]  -8.017594e-03  0.002839 -7.288721e-05  0.000000